# Prompt Engineering with Amazon Bedrock

> *This notebook explores prompt engineering techniques for working with foundation models available in Amazon Bedrock.*

Prompt engineering is a key skill in optimizing interactions with large language models. By crafting effective prompts, you can guide models to produce more accurate, relevant, and reliable outputs for your specific use cases.

### Objectives
- Understand the basics of prompt engineering.
- Learn techniques to create better prompts for tasks such as summarization, question answering, and entity extraction.
- Experiment with Amazon Bedrock models like Titan and Claude.

### Pre-requisites
- Access to Amazon Bedrock.
- Completion of the Bedrock boto3 setup notebook.


## Setup
Let's start by importing the required libraries and configuring Bedrock.

In [ ]:
from dotenv import load_dotenv
import botocore.exceptions
import os
import json
import sys
import boto3

aws_region = "us-east-1"

load_dotenv(".env")

bedrock_runtime_client = boto3.client("bedrock-runtime", region_name=aws_region)
bedrock_management_client = boto3.client('bedrock', region_name=aws_region)
bedrock_agent_client = boto3.client('bedrock-agent', region_name=aws_region)
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime', region_name=aws_region)
cloudformation_client = boto3.client('cloudformation', region_name=aws_region)

boto3.__version__

In [ ]:
context = """
Amazon Bedrock is a new service that makes foundation models accessible via an API. 
It provides a scalable, reliable, and secure way to build generative AI-based applications 
without managing any infrastructure. With Bedrock, you can quickly get started with foundation 
models from various providers, customize them with your data, and integrate them into your applications.
This eliminates the complexities of building and maintaining machine learning models, allowing 
businesses to focus on innovation and accelerating time-to-market for their products. 

Foundation models offered through Bedrock are pre-trained on diverse datasets, making them 
highly capable for various generative AI tasks such as text generation, summarization, 
translation, and content creation. Users can access models from providers like Anthropic, AI21 Labs, 
and Amazon’s own Titan models, giving them a wide range of options tailored to different use cases.

Bedrock also supports easy customization of foundation models using your proprietary data, 
without requiring you to train models from scratch. This feature is particularly valuable for 
organizations that need models to align closely with their domain-specific knowledge and operational needs.

Additionally, Bedrock provides seamless integration into existing workflows and applications, 
with built-in security and compliance features to ensure data privacy and governance. 
Organizations can confidently deploy generative AI capabilities while adhering to industry standards 
and regulations. With Amazon Bedrock, businesses of all sizes can unlock the power of generative AI 
to create innovative solutions and enhance productivity.
"""

## Example 1: Summarization

In this example, we'll explore how prompt design affects the summarization task using the Titan model.

In [ ]:
def get_summary(prompt, parameters, model_id):
    try:
        body = json.dumps(
            {
                "inputText": prompt, 
                "textGenerationConfig": parameters
            }
        )

        response = bedrock_runtime_client.invoke_model(
            body=body,
            modelId=model_id,
            accept="application/json",
            contentType="application/json"
        )
        response_body = json.loads(response.get('body').read())
        summary = response_body.get('results')[0].get('outputText')
        print("Summary:", summary)
    except botocore.exceptions.ClientError as error:
        print("Error invoking the model:", error)

In [ ]:
import ipywidgets as widgets

agent_foundation_model_selector = widgets.Dropdown(
    options=[
        (model['modelName'], model['modelId']) 
        for model in bedrock_management_client.list_foundation_models(
            byProvider="Amazon",
            byOutputModality="TEXT",
            byInferenceType="ON_DEMAND"
        ).get('modelSummaries', []) if "Nova" not in model['modelName'] 
    ],
    value='amazon.titan-tg1-large',
    description='FM:',
    disabled=False,
)
agent_foundation_model_selector

### Experiment with Variations

- Adjust the `temperature` parameter to control the randomness of the output.
- Modify the instruction in the prompt for more detailed or concise summaries.
- Use the `get_summary` function to complete this task.


<details>
<summary>Click here for the solution</summary>
    
```python
prompt = f"""
Please summarize the following text:
{context}
"""

parameters = {
    "maxTokenCount": 200,
    "temperature": 0.7,
    "topP": 1.0
}

model_id = agent_foundation_model_selector.value
```
</details>

## Example 2: Question Answering

Here, we'll design prompts to extract specific answers from the model using Titan and Claude models.

In [ ]:
def get_model_answer(qa_prompt, parameters, model_id):
    try:
        body = json.dumps(
            {
                "inputText": qa_prompt,
                "textGenerationConfig": parameters
            }
        )

        response = bedrock_runtime_client.invoke_model(
            body=body,
            modelId=model_id,
            accept="application/json",
            contentType="application/json"
        )

        response_body = json.loads(response.get('body').read())
        results = response_body.get('results')

        if results and len(results) > 0:
            answer = results[0].get('outputText', "No answer generated.")
        else:
            answer = "No results found in the model response."

        print("Answer:", answer)
        return answer

    except botocore.exceptions.ClientError as error:
        print("Error invoking the model:", error)
        return f"Error: {error.response['Error']['Message']}" if 'Error' in error.response else "An unexpected error occurred."
    except Exception as e:
        print("An unexpected error occurred:", e)
        return "An unexpected error occurred. Please check the input and try again."

In [ ]:
import ipywidgets as widgets

agent_foundation_model_selector = widgets.Dropdown(
    options=[
        (model['modelName'], model['modelId']) 
        for model in bedrock_management_client.list_foundation_models(
            byProvider="Amazon",
            byOutputModality="TEXT",
            byInferenceType="ON_DEMAND"
        ).get('modelSummaries', []) if "Nova" not in model['modelName'] 
    ],
    value='amazon.titan-text-premier-v1:0',
    description='FM:',
    disabled=False,
)
agent_foundation_model_selector

### Experiment with Context
- Add more specific details to the context to observe how the model's answer changes.
- Test the model with ambiguous or incomplete contexts.
- Use the `get_model_answer` function to complete this task.

<details>
<summary>Click here for the solution</summary>
    
```python
question = "What is Amazon Bedrock?"

qa_prompt = f"""
Answer the question based on the context provided below.

Context:
{context}

Question:
{question}

Answer:
"""

model_id = agent_foundation_model_selector.value
```
</details>

## Example 3: Entity Extraction
In this task, we'll guide the model to extract structured information from unstructured text.

In [ ]:
email_text = """
Dear Customer Support Team,

I hope this message finds you well. My name is John Doe, and I am reaching out to inquire about a couple of books that I am interested in purchasing from your store. The first book is titled 'Generative AI: A Comprehensive Guide', which I believe is one of your bestsellers. Could you confirm if this book is currently available in stock? If so, could you also let me know the price and estimated shipping times to New York City?

Additionally, I am also interested in another book, 'Machine Learning for Beginners', which I couldn’t find on your website. Could you check if this book is available and provide the same details as above?

Finally, I would appreciate it if you could inform me about any ongoing discounts or promotions that might apply to these books or other related items.

Looking forward to your response.

Best regards,
John Doe
"""

In [ ]:
def extract_entities(entity_prompt, parameters, model_id):
    try:
        body = json.dumps(
            {
                "inputText": entity_prompt,
                "textGenerationConfig": parameters
            }
        )

        response = bedrock_runtime_client.invoke_model(
            body=body,
            modelId=model_id,
            accept="application/json",
            contentType="application/json"
        )

        response_body = json.loads(response.get('body').read())
        results = response_body.get('results')

        if results and len(results) > 0:
            entities = results[0].get('outputText', "No entities extracted.")
        else:
            entities = "No results found in the model response."

        print("Extracted Entities:", entities)
        return entities

    except botocore.exceptions.ClientError as error:
        print("Error invoking the model:", error)
        return f"Error: {error.response['Error']['Message']}" if 'Error' in error.response else "An unexpected error occurred."
    except Exception as e:
        print("An unexpected error occurred:", e)
        return "An unexpected error occurred. Please check the input and try again."

In [ ]:
import ipywidgets as widgets

agent_foundation_model_selector = widgets.Dropdown(
    options=[
        (model['modelName'], model['modelId']) 
        for model in bedrock_management_client.list_foundation_models(
            byProvider="Amazon",
            byOutputModality="TEXT",
            byInferenceType="ON_DEMAND"
        ).get('modelSummaries', []) if "Nova" not in model['modelName'] 
    ],
    value='amazon.titan-text-premier-v1:0',
    description='FM:',
    disabled=False,
)
agent_foundation_model_selector

### Customizing Entity Extraction
- Experiment with different entity types to extract.
- Use tags or structured formats to make extraction easier.
- Use the `extract_entities` function to complete this task.

<details>
<summary>Click here for the solution</summary>
    
```python
entity_prompt = f"""
Extract the following information from the email:
1. Name of the sender.
2. Book title mentioned.
3. Questions asked.

Email:
{email_text}

Provide the answers in JSON format with keys: 'name', 'book', 'questions'.
"""

parameters = {
    "maxTokenCount": 200,
    "temperature": 0.7,
    "topP": 1.0
}

model_id = agent_foundation_model_selector.value
```
    
</details>

## Prompt Engineering with LangChain AWS

Building on the foundational knowledge of prompt engineering in Amazon Bedrock, this section explores the use of **LangChain AWS**, a high-level framework for seamless integration with Amazon Bedrock. With LangChain AWS, you can simplify interactions with foundation models, enabling rapid experimentation and optimization of generative AI applications.

### Objectives
- Extend prompt engineering techniques to **LangChain AWS** for enhanced flexibility.
- Apply advanced techniques to tasks like summarization, question answering, and entity extraction.
- Experiment with a structured framework to streamline foundation model interactions.

### Why LangChain AWS?
While Amazon Bedrock APIs provide direct access to foundation models, **LangChain AWS** abstracts low-level operations, enabling:
- **Simplified workflows:** Manage prompts and responses efficiently.
- **Custom chaining:** Combine tasks like summarization and entity extraction in a single flow.
- **Model-agnostic flexibility:** Switch between foundation models without major code changes.

### Explanation of Labels in Chat Prompts

In the `ChatPromptTemplate` used within LangChain, labels such as `system`, `human`, and `assistant` serve distinct roles to structure conversations. Here's a quick explanation:

- **`system`**: Represents the role of the AI assistant and provides high-level guidance on its behavior or objective. For example, it can set the context by specifying that the assistant is a summarizer or a general knowledge expert.
  
- **`human`**: Represents input from the user. This is where the specific prompt or question is provided, often using placeholders (like `{context}`) to insert dynamic content at runtime.

- **`assistant`**: Represents responses generated by the AI model. This is typically used to pre-fill examples or specify expected behavior, though it’s optional in many cases.

These labels ensure clarity in conversations and guide the AI to respond appropriately within the defined context.

In [ ]:
import ipywidgets as widgets

agent_foundation_model_selector = widgets.Dropdown(
    options=[
        (model['modelName'], model['modelId']) 
        for model in bedrock_management_client.list_foundation_models(
            byProvider="Anthropic",
            byOutputModality="TEXT",
            byInferenceType="ON_DEMAND"
        ).get('modelSummaries', [])
    ],
    value='anthropic.claude-3-5-sonnet-20240620-v1:0',
    description='FM:',
    disabled=False,
)
agent_foundation_model_selector

In [ ]:
from langchain_aws import ChatBedrock
import json

model_id = agent_foundation_model_selector.value
llm = ChatBedrock(
    model_id=model_id,
    region=aws_region,
    model_kwargs={
        "max_tokens": 300,
        "temperature": 0.7,
        "stop_sequences": ["\n\nHuman:"]
    }
)

## Example 1: Summarization and Translation

In this example, we'll demonstrate how to perform summarization and translation tasks using the LangChain AWS integration with Amazon Bedrock. By leveraging the flexibility of **`ChatPromptTemplate`**, we'll dynamically craft prompts for each task and invoke the Bedrock LLM for concise outputs. 

This approach allows for:
- Efficient summarization of large text passages.
- Dynamic translation between specified languages. 

Let's begin!

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

def get_summarization_chain(llm):
    summarization_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant that summarizes text concisely."),
        ("human", "Please summarize the following text:\n{context}")
    ])

    summarization_chain = summarization_prompt | llm

    return summarization_chain

### Using a Summarization Chain

The code snippet below demonstrates how to use a summarization chain to process a given `context` and generate a summary. You can integrate it into your workflow to automate text summarization tasks.

```python
summarization_chain = [...]   # Initialize your translation chain
inputs = {"context": "<put-your-context-here>"}
response = summarization_chain.invoke(inputs)
print("Summary:", response.content)
```


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

def ger_translation_chain(llm):
    translation_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant that translates {input_language} to {output_language}."),
        ("human", "{input}")
    ])

    translation_chain = translation_prompt | llm

    return translation_chain

### Using a Translation Chain

The code snippet below demonstrates how to use a translation chain to translate text from one language to another. You can integrate it into your workflow to automate translation tasks.

```python
translation_chain = [...]  # Initialize your translation chain
translation_inputs = {
    "input_language": "English",
    "output_language": "German",
    "input": "I love programming."
}
response = translation_chain.invoke(translation_inputs)

print("Translation:", response.content)
```

### Experimentation
- Modify the temperature to observe changes in output style.
- Adjust the max tokens to control the length of the summary.

## Example 2: Question Answering
We'll craft a prompt to retrieve specific information from a given context.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate


def get_question_answering(llm):
    qa_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a knowledgeable assistant answering questions."),
        ("human",
    """
    Answer the question based on the context provided below:

    Context:
    {context}

    Question:
    {question}
    """
        )
    ])

    qa_chain = qa_prompt | llm

    return qa_chain

### Using a QA Chain

The code snippet below demonstrates how to use a question-answering (QA) chain to extract answers from a given context. You can integrate it into your workflow to automate answering questions based on specific text inputs.

```python
qa_chain = [...]  # Initialize your QA chain
question = "What is Amazon Bedrock?"
qa_inputs = {
    "context": "<put-your-context-here>",
    "question": question
}
response = qa_chain.invoke(qa_inputs)

print("Answer:", response.content)
```

### Experimentation
- Provide ambiguous or incomplete contexts to test the model's behavior.
- Add specific details to the context to improve accuracy.

## Example 3: Entity Extraction
Let's guide the model to extract structured information from an unstructured email.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

def get_entity_extraction_chain(llm):

    entity_extraction_prompt = ChatPromptTemplate.from_messages([
        (
            "system", 
            "You are a helpful assistant extracting structured information from emails."
        ),
        (
            "human", 
            """
            Extract the following information from the email:
            - Sender's name.
            - Book title mentioned.
            - Questions asked.

            Email:
            {email_text}

            Provide the answers in JSON format with keys: 'name', 'book', 'questions'.
            """
        )
    ])

    entity_extraction_chain = entity_extraction_prompt | llm

    return entity_extraction_chain

### Using an Entity Extraction Chain

The code snippet below demonstrates how to use an entity extraction chain to identify entities from a given text. You can integrate it into your workflow to automate entity extraction tasks.

```python
entity_extraction_chain = [...]  # Initialize your NER chain
entity_extraction_inputs = {
    "email_text": "<put-your-email-text-here>"
}

response = entity_extraction_chain.invoke(entity_extraction_inputs)

print("Extracted Entities:", response.content)
```

### Experimentation
- Try with different email structures and observe the extracted entities.
- Test for emails with missing or incomplete information.

## Example 4: Combined Chain
In this example, we'll demonstrate how to combine multiple chains to perform summarization, translation, and information extraction sequentially. This approach allows for streamlined processing of text through different tasks, such as summarizing a given context, translating it into another language, and finally extracting structured information.

In [ ]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a knowledgeable assistant answering questions."),
    ("human", "Answer the question based on the context provided below: Context: {context} Question: {question}")
])

entity_extraction_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant extracting structured information. Provide the answers in JSON format with keys: 'key_points', 'models_mentioned', 'benefits'."),
    ("human", "Extract key points, models mentioned, and benefits from the following context: {context}")
])

summarization_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that summarizes text concisely."),
    ("human", "Summarize the following information:\nAnswer: {answer}\nEntities: {entities}")
])

qa_chain = {"context": itemgetter("context"), "question": itemgetter("question")} | qa_prompt | llm | StrOutputParser()
entity_extraction_chain = {"context": itemgetter("context")} | entity_extraction_prompt | llm | StrOutputParser()

summarization_chain = {
    "answer": qa_chain,
    "entities": entity_extraction_chain
} | summarization_prompt | llm | StrOutputParser()

In [ ]:
inputs = {
    "context": context,
    "question": "What is not Amazon Bedrock?"
}

results = summarization_chain.invoke(inputs)

In [ ]:
print(results.strip())

In [ ]:
print(qa_chain.invoke(inputs))

In [ ]:
print(entity_extraction_chain.invoke(inputs))

### Experimentation
- Use various contexts to observe how the combined chain performs for summarization, question answering, and entity extraction.
- Test with incomplete or ambiguous contexts and evaluate the adaptability of the combined chain.
- Experiment with different questions and examine how well the chain handles diverse queries.
- Modify the structure of the context and assess how it impacts the quality of extracted entities and the overall results.

### **Challenge 1: Generate Multi-Lingual FAQs**
**Objective:** Use prompt engineering techniques to create an FAQ page in multiple languages based on the provided context.

**Scenario:**
You are tasked with creating an FAQ section for a website about **Amazon Bedrock**. The FAQ should include a question and a short, concise answer in **English, Spanish, and German**.

**Tasks:**
1. Write a prompt that instructs the model to:
   - Generate a list of 5 FAQs based on the provided context about Amazon Bedrock.
   - Provide answers to each question.
   - Translate the FAQs and answers into Spanish and German.

2. Use **LangChain AWS** or the Bedrock API directly to:
   - Generate the FAQ content.
   - Ensure the translations are accurate and contextually relevant.
   
3. Output the FAQ content in a structured JSON format, like:
   ```json
   [
       {
           "question": "What is Amazon Bedrock?",
           "answer": "Amazon Bedrock is a service that makes foundation models accessible via an API.",
           "translations": {
               "es": "¿Qué es Amazon Bedrock?",
               "de": "Was ist Amazon Bedrock?"
           }
       },
       ...
   ]
   ```

**Bonus:**
- Include a summarization of the FAQ content in **French** at the end.

### **Challenge 2: Extract and Summarize Key Benefits for a Report**
**Objective:** Extract and summarize the key benefits of Amazon Bedrock from the given context to create a concise report.

**Scenario:**
You need to prepare a report highlighting the **key benefits** of Amazon Bedrock for stakeholders. The report should include:
- A list of 3-5 bullet points summarizing the benefits.
- Structured extraction of models mentioned in the context.
- A short summary of the key benefits for an executive audience.

**Tasks:**
1. Write a prompt that:
   - Extracts the **key benefits** of Amazon Bedrock.
   - Identifies and lists the foundation models mentioned in the context.
   - Summarizes the benefits into a 2-3 sentence paragraph.

2. Use **LangChain AWS** to:
   - Extract and structure the benefits and models into a JSON format like:
     ```json
     {
         "benefits": [
             "Easy integration into workflows",
             "Wide range of foundation models",
             "Customizable with proprietary data"
         ],
         "models": [
             "Titan",
             "Claude",
             "Stable Diffusion"
         ],
         "summary": "Amazon Bedrock provides seamless integration of generative AI capabilities into workflows, offers diverse foundation models, and supports customization with proprietary data."
     }
     ```
   
3. Generate the content with clear formatting for inclusion in a report.

**Bonus:**
- Include a follow-up chain to translate the summary into **Japanese**.

## Summary

Prompt engineering, combined with the powerful capabilities of LangChain AWS and Amazon Bedrock models, allows you to enhance the effectiveness of foundation models for a wide range of generative AI tasks. By crafting clear prompts and experimenting with different configurations, you can tailor model responses to suit your specific use cases seamlessly.

### Key Takeaways
- Use clear and specific instructions in your prompts to guide the model effectively.
- Experiment with temperature, token count, and top-p parameters to optimize performance.
- Leverage LangChain's flexible API for seamless integration and streamlined interactions with foundation models.
- Iterate on your prompts and configurations based on the model's output to refine and improve results.
- Combine generative AI capabilities with LangChain AWS to unlock innovative solutions tailored to your needs.